In [ ]:
# Import libraries for LLM APIs, web scraping, environment management, and UI.
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [ ]:
# Import Gradio for building the chatbot user interface.
import gradio as gr # oh yeah!

In [ ]:
# Load environment variables and print API key prefixes for debugging.
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Initialize OpenAI, Anthropic, and Google Generative AI clients.
openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [ ]:
# Define the system message for the chatbot's personality and response style.
system_message = "You are a snarky but helpful assistant that responds in markdown."

In [ ]:
# Function to stream responses from OpenAI GPT model using the system message and user prompt.
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
# Function to stream responses from Anthropic Claude model using the system message and user prompt.
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
# Function to stream responses from Google Gemini model using the system message and user prompt.
def stream_gemini(prompt):
    model = google.generativeai.GenerativeModel("gemini-pro")
    response = model.generate_content(
        [system_message, prompt],
        generation_config={"temperature": 0.7, "max_output_tokens": 1000},
        stream=True
    )
    result = ""
    for chunk in response:
        result += getattr(chunk, "text", "") or ""
        yield result

In [ ]:
# Function to stream responses from the selected model (GPT, Claude, or Gemini) based on user input.
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
# Create and launch the Gradio interface for the chatbot, allowing users to select a model and interact.
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()